## 2.1 Realizácia predspracovania dát (5b).

- (A-1b) Dáta si rozdeľte na trénovaciu a testovaciu množinu podľa vami preddefinovaného pomeru. Ďalej pracujte len **s trénovacím datasetom**.


- (B-1b) Transformujte dáta na vhodný formát pre ML t.j. jedno pozorovanie musí byť opísané jedným riadkom a každý atribút musí byť v numerickom formáte (encoding). Iteratívne integrujte aj kroky v predspracovaní dát z prvej fázy (missing values, outlier detection) ako celok. 


- (C-2b) Transformujte atribúty dát pre strojové učenie podľa dostupných techník minimálne: scaling (2 techniky), transformers (2 techniky) a ďalšie. Cieľom je aby ste testovali efekty a vhodne kombinovali v dátovom pipeline (od časti 2.3 a v 3. fáze). 

- (D-1b) Zdôvodnite Vaše voľby/rozhodnutie pre realizáciu (t.j. zdokumentovanie)



# !!!!!! TODO !!!!!! TOTO ESTE SKONTROLOVAT A PREJST/PREPISAT

# Zdokumentovanie a Zdôvodnenie Rozhodnutí pri Transformácii Atribútov

Pri transformácii dát pre strojové učenie je dôležité zabezpečiť, aby atribúty mali vhodnú škálu, boli stabilné a aby ich distribúcia vyhovovala potrebám modelov. Nižšie je uvedené zdôvodnenie zvolených techník transformácie a škálovania.

## 1. Scaling (Škálovanie)
Výber techník škálovania bol motivovaný snahou o optimalizáciu výkonnosti modelov, ktoré často očakávajú normalizované alebo štandardizované vstupy.

### 1.1 StandardScaler
- **Dôvod**: StandardScaler bol zvolený, pretože štandardizuje atribúty na priemer 0 a štandardnú odchýlku 1, čo je vhodné pre algoritmy citlivé na rozsah dát, ako sú SVM, k-means alebo lineárna regresia.
- **Výhoda**: Táto metóda udržuje štatistickú vlastnosť rozdelenia dát a dobre funguje pre dáta, ktoré už majú približne normálne rozdelenie.

### 1.2 MinMaxScaler
- **Dôvod**: MinMaxScaler bol vybraný pre prípady, kde bolo nutné previesť dáta do jednotného rozsahu, napríklad 0 až 1, čo je dôležité pri metódach, ako sú neurónové siete, kde normalizácia atribútov zlepšuje konvergenciu.
- **Výhoda**: Škálovanie do konkrétneho rozsahu môže zvýšiť robustnosť algoritmu voči outlierom a zlepšiť interpretovateľnosť hodnoty každého atribútu.

## 2. Transformers (Transformácie)
Transformácie sme zvolili na stabilizáciu rozdelenia dát a odstránenie šikmosti (skewness), aby sa modely mohli lepšie zamerať na relevantné vzory.

### 2.1 PowerTransformer
- **Dôvod**: PowerTransformer bol zvolený pre svoje schopnosti stabilizovať rozdelenie dát pomocou Yeo-Johnson transformácie. Táto transformácia sa odporúča, ak sú atribúty pozitívne aj negatívne, pričom transformácia stabilizuje šikmosť a robí rozdelenie viac normálnym.
- **Výhoda**: Zlepšenie normality môže pomôcť modelom lineárneho typu a optimalizáciám, ktoré sa opierajú o normálne rozdelenie dát, ako sú regresné modely a PCA.

### 2.2 QuantileTransformer
- **Dôvod**: QuantileTransformer bol zvolený pre svoju schopnosť transformovať rozdelenie dát na normálnu distribúciu bez ohľadu na pôvodné rozdelenie, čo je užitočné pri atribútoch s výraznými odchýlkami.
- **Výhoda**: Táto metóda môže efektívne normalizovať dátové body a je vhodná pre modely citlivé na rozdelenie dát, ako sú algoritmy KNN alebo Bayesovské modely.

## 3. Kombinácia Transformácií v Pipeline
Pipeline bola zostavená na jednoduchšie testovanie a iteratívne kombinácie rôznych techník. Kombinovanie krokov zjednodušuje prácu a zabezpečuje konzistentnú predprípravu dát pre modelovanie.

- **Výhoda pipeline**: Pipeline podporuje opakovateľnosť a konzistenciu v prístupe k predspracovaniu dát. Týmto spôsobom môžeme efektívne otestovať rôzne kombinácie transformácií, vybrať najlepšie varianty a následne ich jednoducho aplikovať na nové dáta alebo testovaciu množinu.

## Celkové zdôvodnenie
Tieto voľby boli zamerané na optimalizáciu dát pre rôzne modely strojového učenia, od lineárnych metód po algoritmy citlivé na rozsah a rozdelenie dát. Skúmanie viacerých techník nám umožňuje prispôsobiť transformáciu atribútov tak, aby maximalizovala výkonnosť a presnosť modelu, čo je cieľom tejto fázy prípravy dát.


## 2.2 Výber atribútov pre strojové učenie (5b)

- (A-3b) Zistite, ktoré atribúty (features) vo vašich dátach pre ML sú informatívne k predikovanej premennej (minimálne 3 techniky s porovnaním medzi sebou). 


- (B-1b) Zoraďte zistené atribúty v poradí podľa dôležitosti. 


- (C-1b) Zdôvodnite Vaše voľby/rozhodnutie pre realizáciu (t.j. zdokumentovanie)



## 2.3 Replikovateľnosť predspracovania (5b)

- (A-3b) Upravte váš kód realizujúci predspracovanie trénovacej množiny tak, aby ho bolo možné bez ďalších úprav znovu použiť **na predspracovanie testovacej množiny** v kontexte strojového učenia.


- (B-2b) Využite možnosti **sklearn.pipeline**